In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
plt.style.use('fivethirtyeight')
import seaborn as sns


In [ ]:
df=pd.read_csv('../input/tesla-stock-data-from-2010-to-2020/TSLA.csv')

In [ ]:
df=df.set_index(pd.DatetimeIndex(df['Date'].values))
df=df.tail(500)
df

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(12.5,4.5))
plt.plot(df['Adj Close'],label='Close Price')
plt.title('TESLA Adj Close Price')
plt.xlabel('Date')
plt.ylabel('Price(USD)')
plt.legend()
plt.show()

In [ ]:
ma30=pd.DataFrame()
ma30['AM']=df['Adj Close'].rolling(window=30).mean()
ma100=pd.DataFrame()
ma100['AM']=df['Adj Close'].rolling(window=100).mean()

In [ ]:
data=pd.DataFrame()
data['TESLA']=df['Adj Close']
data['MA30']=ma30['AM']
data['MA100']=ma100['AM']

In [ ]:
 def signal(data):
        signalBuy=[]
        signalSell=[]
        f=-1
        for i in range(len(data)):
            if data['MA30'][i]>data['MA100'][i]:
                if f != 1:
                    signalBuy.append(data['TESLA'][i])
                    signalSell.append(np.nan)
                    f=1
                else:
                    signalBuy.append(np.nan)
                    signalSell.append(np.nan)

            elif data['MA30'][i]< data['MA100'][i]:    
                if  f!= 0:
                    signalBuy.append(np.nan)
                    signalSell.append(data['TESLA'][i])
                    f=0
                else:
                    signalBuy.append(np.nan)
                    signalSell.append(np.nan)
            else :
                signalBuy.append(np.nan)
                signalSell.append(np.nan)
                
        return signalBuy, signalSell

In [ ]:
buy_sell=signal(data)
data['buy signal']=buy_sell[0]
data['sell signal']=buy_sell[1]

In [ ]:
data

In [ ]:

plt.figure(figsize=(16,8))
plt.plot(data['TESLA'],label='TESLA')
plt.plot(data['MA30'],label='MA30')
plt.plot(data['MA100'],label='MA100')
plt.scatter(data.index,data['buy signal'],label='BUY',marker='^',color='g')
plt.scatter(data.index,data['sell signal'],label='SELL',marker='.',color='r')
plt.title('two moving average signal')
plt.xlabel('DATE')
plt.ylabel('PRICE(USD)')
plt.legend()
plt.show()


**MFI andicator**

In [ ]:
plt.figure(figsize=(12.5,4.5))
plt.plot(df['Close'],label='Close Price')
plt.title('TESLA Close Price')
plt.xlabel('Date')
plt.ylabel('Price(USD)')
plt.legend()
plt.show()

In [ ]:
typical_price=(df['Close']+df['High']+df['Low'])/3

In [ ]:
period=14
money_flow=typical_price*df['Volume']
money_flow

In [ ]:
positive_flow=[]
negative_flow=[]
for i in range(1,len(typical_price)):
    if typical_price[i]>typical_price[i-1]:
        positive_flow.append(money_flow[i-1])
        negative_flow.append(0)
    elif typical_price[i]< typical_price[i-1]:
        positive_flow.append(0)
        negative_flow.append(money_flow[i-1])
    else:
        positive_flow.append(0)
        negative_flow.append(0)
    

In [ ]:
positive_mf=[]
negative_mf=[]
for i in range(period-1, len(positive_flow)):
    positive_mf.append(sum(positive_flow[i+1-period:i+1]))
for i in range(period-1, len(negative_flow)):
    negative_mf.append(sum(negative_flow[i+1-period:i+1]))   

In [ ]:
#MFI equation
mfi=100*(np.array(positive_mf)/(np.array(positive_mf)+np.array(negative_mf)))
mfi

In [ ]:
df1=pd.DataFrame()
df1['MFI']=mfi

In [ ]:
plt.figure(figsize=(12.5,4.5))
plt.plot(df1['MFI'],label='MFI')
plt.axhline(20,linestyle='--', color='r')
plt.axhline(30,linestyle='--', color='b')
plt.axhline(70,linestyle='--', color='b')

plt.axhline(80,linestyle='--', color='r')
plt.title('MFI')
plt.xlabel('Date')
plt.ylabel('MFI')
plt.legend()
plt.show()

In [ ]:
fig,(ax1,ax2)=plt.subplots(nrows=2,ncols=1,figsize=(12.5,6))
ax1.plot(df['Close'],label='Close Price')
ax2.plot(df1['MFI'],label='MFI')
ax2.axhline(20,linestyle='--', color='r', alpha=0.5)
ax2.axhline(30,linestyle='--', color='b',alpha=0.5)
ax2.axhline(70,linestyle='--', color='b',alpha=0.5)
ax2.axhline(80,linestyle='--', color='r',alpha=0.5)
ax1.set_title('MFI Visualizer')